In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import vectorbt as bt
import pandas_ta as ta
import yfinance as yf
import decimal

ModuleNotFoundError: No module named 'vectorbt'

In [10]:
data = yf.download('BTC-USD', start='2024-12-15', end='2024-12-31', interval='15m')
data = data.asfreq('15min')
print(data.head())
data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
data.drop(columns=['Volume'], inplace=True)


data_1h = yf.download('BTC-USD', start='2024-12-15', end='2024-12-31', interval='1h')
data_1h = data_1h.asfreq('1h')
data_1h.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
data_1h.drop(columns=['Volume'], inplace=True)
data_1h.head()

data.index = data.index.tz_localize(None)
data_1h.index = data_1h.index.tz_localize(None)


data_1h.head()


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Price                        Close           High            Low  \
Ticker                     BTC-USD        BTC-USD        BTC-USD   
Datetime                                                           
2024-12-15 00:00:00  101453.625000  101475.960938  101355.429688   
2024-12-15 00:15:00  101484.734375  101511.328125  101411.828125   
2024-12-15 00:30:00  101337.343750  101512.976562  101227.953125   
2024-12-15 00:45:00  101416.492188  101416.492188  101301.031250   
2024-12-15 01:00:00  101459.796875  101459.796875  101413.109375   

Price                         Open    Volume  
Ticker                     BTC-USD   BTC-USD  
Datetime                                      
2024-12-15 00:00:00  101393.679688         0  
2024-12-15 00:15:00  101469.421875  43577344  
2024-12-15 00:30:00  101512.976562     28672  
2024-12-15 00:45:00  101332.328125  28483584  
2024-12-15 01:00:00  101414.109375         0  


,Close,High,Low,Open
Datetime,,,,
2024-12-15 00:00:00,101416.492188,101512.976562,101227.953125,101393.679688
2024-12-15 01:00:00,102096.843750,102124.218750,101413.109375,101414.109375
2024-12-15 02:00:00,101820.156250,102097.320312,101746.921875,102097.320312
2024-12-15 03:00:00,101733.476562,101889.976562,101705.804688,101794.437500
2024-12-15 04:00:00,102280.882812,102731.304688,101697.937500,101697.937500


In [251]:
def pattern_hammer_bull(df: pd.DataFrame):

    df['50_percentile'] = (df['High'] + df['Low']) / 2
    df['90_percentile'] = df['Low'] + 0.9 * (df['High'] - df['Low'])

    # Define the hammer condition
    condition = (
        (df['Open'] >= df['50_percentile']) &  # Open in the upper half
        (df['Close'] >= df['90_percentile']) &  # Close near the top
        (df['Close'] >= df['Open'])  # Bullish candlestick
    )

    # Assign hammer flag
    df['hammer_bull'] = np.where(condition, 1, 0)
    return df


In [252]:
def pattern_hammer_bear(df: pd.DataFrame):
    # Calculate percentiles
    df['50_percentile'] = (df['High'] + df['Low']) / 2
    df['10_percentile'] = df['Low'] + 0.1 * (df['High'] - df['Low'])  # 10th percentile

    # Define the bearish hammer condition
    condition = (
        (df['Open'] <= df['50_percentile']) &  # Open in the lower half
        (df['Close'] <= df['10_percentile']) &  # Close near the bottom
        (df['Close'] <= df['Open'])  # Bearish candlestick
    )

    # Assign hammer flag
    df['hammer_bear'] = np.where(condition, 1, 0)
    return df


In [253]:
data = pattern_hammer_bull(data)
data = pattern_hammer_bear(data)
data.drop(columns=['50_percentile', '90_percentile', '10_percentile'], inplace=True)
data.head(20)

,Close,High,Low,Open,hammer_bull,hammer_bear
Datetime,,,,,,
2024-12-15 00:00:00,101453.625000,101475.960938,101355.429688,101393.679688,0,0
2024-12-15 00:15:00,101484.734375,101511.328125,101411.828125,101469.421875,0,0
2024-12-15 00:30:00,101337.343750,101512.976562,101227.953125,101512.976562,0,0
2024-12-15 00:45:00,101416.492188,101416.492188,101301.031250,101332.328125,0,0
2024-12-15 01:00:00,101459.796875,101459.796875,101413.109375,101414.109375,0,0
2024-12-15 01:15:00,101532.890625,101578.039062,101466.445312,101466.445312,0,0
2024-12-15 01:30:00,101736.195312,101736.195312,101519.734375,101568.414062,0,0
2024-12-15 01:45:00,102096.843750,102124.218750,101803.210938,101803.210938,0,0
2024-12-15 02:00:00,101958.007812,102097.320312,101901.507812,102097.320312,0,0


In [254]:
adx_results = ta.adx(data_1h['High'], data_1h['Low'], data_1h['Close'], length=5)

# Assign to DataFrame
data_1h['adx'] = adx_results['ADX_5']  # ADX values

In [255]:
data_1h.describe()

,Close,High,Low,Open,adx
count,384.000000,384.000000,384.000000,384.000000,375.000000
mean,98252.037394,98596.031392,97904.038391,98267.534546,40.458429
std,4228.177339,4234.495881,4214.562456,4225.667052,14.927328
min,91863.742188,92013.781250,91347.031250,91530.390625,13.687296
25%,94825.537109,95056.720703,94662.369141,94861.589844,28.747666
50%,97101.683594,97391.253906,96621.410156,97084.828125,37.874722
75%,101672.382812,101880.531250,101361.154297,101644.939453,50.724990
max,108240.085938,108240.085938,107158.226562,108239.414062,86.965356


In [256]:
data["adx"] = data_1h["adx"].reindex(data.index, method="ffill")
data.dropna(inplace=True)

print(data['hammer_bear'].value_counts())
print(data['hammer_bull'].value_counts())
data.head(15)

hammer_bear
0    1439
1      61
Name: count, dtype: int64
hammer_bull
0    1438
1      62
Name: count, dtype: int64


,Close,High,Low,Open,hammer_bull,hammer_bear,adx
Datetime,,,,,,,
2024-12-15 09:00:00,102080.968750,102105.859375,101927.664062,101971.054688,0,0,34.092763
2024-12-15 09:15:00,102006.062500,102040.054688,101959.695312,102008.828125,0,0,34.092763
2024-12-15 09:30:00,102081.085938,102082.304688,101989.367188,101995.882812,0,0,34.092763
2024-12-15 09:45:00,102196.343750,102197.859375,102122.781250,102168.007812,1,0,34.092763
2024-12-15 10:00:00,102315.765625,102384.335938,102194.890625,102194.890625,0,0,38.076058
2024-12-15 10:15:00,102205.375000,102380.304688,102205.375000,102322.875000,0,0,38.076058
2024-12-15 10:30:00,102205.570312,102214.750000,102145.406250,102204.593750,0,0,38.076058
2024-12-15 10:45:00,102278.929688,102293.203125,102206.242188,102206.242188,0,0,38.076058
2024-12-15 11:00:00,102251.539062,102253.414062,102209.593750,102253.414062,0,0,43.752905


In [263]:
data['long_entry'] = ((data['hammer_bull'] == 1) & (data['adx']>35)).astype(int)
data['short_entry'] = ((data['hammer_bear'] == 1) & (data['adx']>35)).astype(int)

col = ['hammer_bull', 'hammer_bear', 'long_entry', 'short_entry']

for i in col:
    print(data[i].value_counts())

data


hammer_bull
0    1438
1      62
Name: count, dtype: int64
hammer_bear
0    1439
1      61
Name: count, dtype: int64
long_entry
0    1463
1      37
Name: count, dtype: int64
short_entry
0    1460
1      40
Name: count, dtype: int64


,Close,High,Low,Open,hammer_bull,hammer_bear,adx,long_entry,short_entry
Datetime,,,,,,,,,
2024-12-15 09:00:00,102080.968750,102105.859375,101927.664062,101971.054688,0,0,34.092763,0,0
2024-12-15 09:15:00,102006.062500,102040.054688,101959.695312,102008.828125,0,0,34.092763,0,0
2024-12-15 09:30:00,102081.085938,102082.304688,101989.367188,101995.882812,0,0,34.092763,0,0
2024-12-15 09:45:00,102196.343750,102197.859375,102122.781250,102168.007812,1,0,34.092763,0,0
2024-12-15 10:00:00,102315.765625,102384.335938,102194.890625,102194.890625,0,0,38.076058,0,0
...,...,...,...,...,...,...,...,...,...
2024-12-30 22:45:00,92658.445312,92761.015625,92557.187500,92736.359375,0,0,40.451902,0,0
2024-12-30 23:00:00,92336.476562,92598.750000,92293.414062,92598.750000,0,0,38.914949,0,0
2024-12-30 23:15:00,92455.960938,92471.828125,92262.507812,92325.296875,0,0,38.914949,0,0


In [264]:
#Portfolio Construction
pf = bt.Portfolio.from_signals(data['Close'], entries=data['long_entry'], short_entries=data['short_entry'],fees=0.001, slippage=0.001, sl_stop=0.02, tp_stop=0.04, upon_opposite_entry='Ignore')

In [265]:
pf.stats()

Start                         2024-12-15 09:00:00
End                           2024-12-30 23:45:00
Period                           15 days 15:00:00
Start Value                                 100.0
End Value                               94.753971
Total Return [%]                        -5.246029
Benchmark Return [%]                    -9.243283
Max Gross Exposure [%]                      100.0
Total Fees Paid                          2.235644
Max Drawdown [%]                        15.975128
Max Drawdown Duration            13 days 09:00:00
Total Trades                                   12
Total Closed Trades                            11
Total Open Trades                               1
Open Trade PnL                           1.688489
Win Rate [%]                            27.272727
Best Trade [%]                           4.970359
Worst Trade [%]                         -2.657008
Avg Winning Trade [%]                    4.255191
Avg Losing Trade [%]                    -2.431417


In [266]:
#Plots
pf.plot(
    subplots=['trades', 'orders', 'trade_pnl', 'cum_returns']
    )

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2828628f-4c52-48e6-9744-0ab03b9093ce',
              'x': array([datetime.datetime(2024, 12, 15, 9, 0),
                          datetime.datetime(2024, 12, 15, 9, 15),
                          datetime.datetime(2024, 12, 15, 9, 30), ...,
                          datetime.datetime(2024, 12, 30, 23, 15),
                          datetime.datetime(2024, 12, 30, 23, 30),
                          datetime.datetime(2024, 12, 30, 23, 45)], dtype=object),
              'xaxis': 'x',
              'y': array([102080.96875  , 102006.0625   , 102081.0859375, ...,  92455.9609375,
                           92498.28125  ,  92645.3359375]),
              'yaxis': 'y'},
             {'customdata': array([[0, 0, 0.0009730177252293398, 0.09990009990008275, 'Long'],
                      